## Optimizing a Pipeline in Azure - Daniel Webb - 20th September 2022

Note: Code from the Azure ML sample Jupyter Notebook below are used:  
```Samples\1.43.0\how-to-use-azureml\ml-frameworks\keras\train-hyperparameter-tune-deploy-with-keras\train-hyperparameter-tune-deploy-with-keras.ipynb```

## Set up workspace

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-208099
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-208099


## Set up compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "StandardD2CPUCluster"

# DW: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive run of the logistic regression model in train.py

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# DW: Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 1), # Inverse of regularization strength
    "--max_iter": choice(50, 100, 150, 200) # Maximum number of iterations to converge
    })

# DW: Specify a Termination Policy (for Early Stopping)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# DW: Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# DW: Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [5]:
# DW: Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3fea831a-227a-46e6-b6f1-a0851eed65b8
Web View: https://ml.azure.com/runs/HD_3fea831a-227a-46e6-b6f1-a0851eed65b8?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-208099/workspaces/quick-starts-ws-208099&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-20T21:55:01.417671][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-20T21:55:02.2727144Z][SCHEDULER][INFO]Scheduling job, id='HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_0' 
[2022-09-20T21:55:02.4143948Z][SCHEDULER][INFO]Scheduling job, id='HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_1' 
[2022-09-20T21:55:02.5324194Z][SCHEDULER][INFO]Scheduling job, id='HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_2' 
[2022-09-20T21:55:02.585217][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-09-20T21:55:02.6745847Z][SCHEDULER][INFO]Scheduling job, id='HD_3fea831a-227a-46e6-b6f1-a0851eed65

{'runId': 'HD_3fea831a-227a-46e6-b6f1-a0851eed65b8',
 'target': 'StandardD2CPUCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-20T21:55:00.624649Z',
 'endTimeUtc': '2022-09-20T22:14:14.506004Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '606d047d-3b82-4366-9f4f-f62b9b3bc45a',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'tele

In [7]:
import joblib
# Get your best run and save the model from that run.

# DW: First, get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric() # The best run based on the primary metric

print(best_run.id) # Display the ID of the best run
print(best_run.get_metrics()) # Display the parameters and primary metric values of the best run

HD_3fea831a-227a-46e6-b6f1-a0851eed65b8_4
{'Regularization Strength:': 0.6914687720101343, 'Max iterations:': 200, 'Accuracy': 0.9099295945617868}


In [8]:
best_run.get_file_names() # DW: Next, identify the file names for the best run

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-09-20-22',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [9]:
# DW: Finally, register the model of the best run to save it
hd_model = best_run.register_model(model_name = 'hd_model',
                                   model_path = 'outputs/model.joblib')

## AutoML run on the same dataset

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# DW: Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dsurl = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(dsurl)

In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [12]:
# DW: Let's first take a look at the data.
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [13]:
# DW: And now see what the output results look like.
y.value_counts()

0    29258
1     3692
Name: y, dtype: int64

So we have that many numerical field are present, and we want to use them to predict a 0 or 1 output. As such, this is a classification problem, just as was done with the logistic regression within the `train.py` script.

As the AutoML config will use just one dataset, `training_data`, we need to merge in the target column, as the cleaning process from the Udacity provided code has split the data into `x` with the various fields, and `y` with the target classification (0 or 1) that we want to predict. This target field can then be referred to as the `label_column_name` of the `training_data`.

Note that as per the below link, we do not need to split the data into training and testing sets as the AutoML will do this automatically, using a method depending on the number of rows of data.

https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits

In [23]:
# DW: Create a new column in dataframe x to store the target y values.
x['y'] = y
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


AutoML can only use a Pandas dataframe as the dataset if it is running locally. If it is running on a compute target, such as the compute cluster specified in this Jupyter Notebook, then the dataset must be coverted to a tabular dataset.  
Ref: https://knowledge.udacity.com/questions/469948

We can do this using `TabularDatasetFactory.register_pandas_dataframe` as detailed in the below link:  
https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.data.dataset_factory.tabulardatasetfactory?view=azure-ml-py

Or we can export the data to CSV and then upload it to a datastore in the workspace.

In [55]:
# List the datastores in the workspace
for datastore in ws.datastores:
    print(datastore)

print()
print('Default datastore: ',ws.get_default_datastore().name)

2022-09-20:23:21:02,228 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f7d9e422d00>
2022-09-20:23:21:02,526 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f7d9e422d00>


datastore
workspaceworkingdirectory
workspaceartifactstore
workspacefilestore
workspaceblobstore

Default datastore:  workspaceblobstore


In [ ]:
# Store the default datastore in a variable
datastore = ws.get_default_datastore()

# Save the Pandas dataframe as a CSV file within the training folder
x.to_csv('training/bankmarketing-train.csv')

# Upload the training folder to the datastore
datastore.upload(src_dir='training', target_path='training', overwrite=True)

# Create our dataset object for the AutoML run
dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,
                                                            ('training/bankmarketing-train.csv'))])

2022-09-20:23:48:40,480 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f7d9e422d00>
2022-09-20:23:48:41,128 INFO     [azure_storage_datastore.py:789] Called AzureBlobDatastore.upload
2022-09-20:23:48:41,142 INFO     [azure_storage_datastore.py:370] Uploading an estimated of 1 files


Uploading an estimated of 1 files


2022-09-20:23:48:41,394 INFO     [azure_storage_datastore.py:370] Uploading training/bankmarketing-train.csv
2022-09-20:23:48:41,395 INFO     [azure_storage_datastore.py:370] Uploaded training/bankmarketing-train.csv, 1 files out of an estimated total of 1
2022-09-20:23:48:41,396 INFO     [azure_storage_datastore.py:370] Uploaded 1 files
2022-09-20:23:48:41,397 INFO     [azure_storage_datastore.py:799] Finished AzureBlobDatastore.upload with count=1.


Uploading training/bankmarketing-train.csv
Uploaded training/bankmarketing-train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [61]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cpu_cluster)

NameError: name 'dataset' is not defined

In [33]:
# DW: Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_er

In [ ]:
# DW: Retrieve and save your best automl model.

automl_best_run = automl_run.get_best_run_by_primary_metric() # The best run based on the primary metric
print(automl_best_run.get_metrics()) # Display the parameters and the primary metric values of the best run

## Tidy up resources

In [54]:
# End the experiment logging that was started in the first code cell
run.complete()

# Finally, delete the CPU cluster
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

